# Исследование надёжности заёмщиков — анализ банковских данных

## Цель исследования:

На основе статистики о платёжеспособности клиентов исследовать влияет ли семейное положение и количество детей клиента на факт возврата кредита в срок

In [1]:
import pandas as pd

In [2]:
data = pd.read_csv('Downloads/data.csv')

In [3]:
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу


In [4]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


In [5]:
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

In [6]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

In [7]:
data['days_employed'] = data['days_employed'].abs()

In [12]:
(data.groupby('income_type')['days_employed'].agg('median')).round(1)

income_type
безработный        366413.7
в декрете            3296.8
госслужащий          2689.4
компаньон            1547.4
пенсионер          365213.3
предприниматель       520.8
сотрудник            1574.2
студент               578.8
Name: days_employed, dtype: float64

In [13]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

In [15]:
data = data[(data['children'] != -1) & (data['children'] != 20)]
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

In [16]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [17]:
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

In [18]:
data['total_income'] = data['total_income'].astype(int)

In [19]:
data['education'] = data['education'].str.lower()

In [20]:
data.duplicated().sum()

71

In [21]:
data = data.drop_duplicates()

In [22]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [23]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

In [24]:
data['purpose'].unique()

array(['покупка жилья', 'приобретение автомобиля',
       'дополнительное образование', 'сыграть свадьбу',
       'операции с жильем', 'образование', 'на проведение свадьбы',
       'покупка жилья для семьи', 'покупка недвижимости',
       'покупка коммерческой недвижимости', 'покупка жилой недвижимости',
       'строительство собственной недвижимости', 'недвижимость',
       'строительство недвижимости', 'на покупку подержанного автомобиля',
       'на покупку своего автомобиля',
       'операции с коммерческой недвижимостью',
       'строительство жилой недвижимости', 'жилье',
       'операции со своей недвижимостью', 'автомобили',
       'заняться образованием', 'сделка с подержанным автомобилем',
       'получение образования', 'автомобиль', 'свадьба',
       'получение дополнительного образования', 'покупка своего жилья',
       'операции с недвижимостью', 'получение высшего образования',
       'свой автомобиль', 'сделка с автомобилем',
       'профильное образование', 'высшее об

In [25]:
def categorize_purpose(row):
    try:
        if 'автом' in row:
            return 'операции с автомобилем'
        elif 'жил' in row or 'недвиж' in row:
            return 'операции с недвижимостью'
        elif 'свад' in row:
            return 'проведение свадьбы'
        elif 'образов' in row:
            return 'получение образования'
    except:
        return 'нет категории'

In [26]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [27]:
children_amount = data.groupby('children').agg({'debt': ['count', 'mean']})
children_amount.columns = ['Кол-во заемщиков', '% невозврата']
sort_children_amount = children_amount.sort_values(by='% невозврата')
sort_children_amount.style.format({'% невозврата': '{:.2%}'})


,Кол-во заемщиков,% невозврата
children,,
5,9,0.00%
0,14091,7.54%
3,330,8.18%
1,4808,9.23%
2,2052,9.45%
4,41,9.76%


**Вывод:** из предоставленных данных видно, что с количеством детей растет % невозврата. Отклонение среди заемщиков с тремя и пятью детьми скорее всего связано с недостаточным количеством данных.

**Вывод:** из предоставленных данных видно, что с *количеством детей* растет *% невозврата*. Отклонение среди заемщиков с тремя и пятью детьми скорее всего связано с недостаточным количеством данных.

In [29]:
depend_family_status = data.groupby('family_status').agg({'debt': ['count', 'mean']})
depend_family_status.columns = ['Кол-во заемщиков', '% невозврата']
sort_depend_family_status = depend_family_status.sort_values(by='% невозврата')
sort_depend_family_status.style.format({'% невозврата': '{:.2%}'})


,Кол-во заемщиков,% невозврата
family_status,,
вдовец / вдова,951,6.62%
в разводе,1189,7.06%
женат / замужем,12261,7.56%
гражданский брак,4134,9.31%
Не женат / не замужем,2796,9.76%


**Вывод:** хорошо видна зависимогсть между ссемейным положением и возвратом кредита в срок. Люди которые не были в официальных отношениях возвращают кредиты в срок реже.

In [30]:
depend_income_category = data.groupby('total_income_category').agg({'debt': ['count', 'mean']})
depend_income_category.columns = ['Кол-во заемщиков', '% невозврата']
sort_depend_income_category = depend_income_category.sort_values(by='% невозврата')
sort_depend_income_category.style.format({'% невозврата': '{:.2%}'})

,Кол-во заемщиков,% невозврата
total_income_category,,
D,349,6.02%
B,5014,7.06%
A,25,8.00%
C,15921,8.50%
E,22,9.09%


**Вывод:** Неудивительно, что наибольший % невозврата кредита у категории с самым низким уровнем дохода, однако обртаное утверждение неверно, категория с самым высоким доходом не имеет самый низкий % невозврата кредита. Однако смущает небольшое количество заемщиков в этих категориях, можем ли мы делать выводы на основании выборки из пары десятков человек. Рассматривая категории с большим количсевтвом заемщиков, зависимость между высоким уровнем дохода и низким % невозврата соблюдается только в категориях 'B' и 'C'.

In [31]:
depend_purpose = data.groupby('purpose_category').agg({'debt': ['count', 'mean']})
depend_purpose.columns = ['Кол-во заемщиков', '% невозврата']
sort_depend_purpose = depend_purpose.sort_values(by='% невозврата')
sort_depend_purpose.style.format({'% невозврата': '{:.2%}'})

,Кол-во заемщиков,% невозврата
purpose_category,,
операции с недвижимостью,10751,7.26%
проведение свадьбы,2313,7.91%
получение образования,3988,9.25%
операции с автомобилем,4279,9.35%


**Вывод:** наименьший % невозврата имеют операции с недвижимостью, наибольший — операции с автомобилем, ненамного отстает категория целей связанная с образованием, а вот проведение свадем напротив стремится к наименьшему % невозврата. Все таки покупка недвижимости для многих взвешанный шаг.